<a href="https://colab.research.google.com/github/hl5chen/ECE1512-2022F-ProjectRepo_YiyangShi_HsuanlingChen/blob/main/Project_B_Supp/ProjectB_NEW_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.4 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB

/content/drive/MyDrive/Colab Notebooks/ECE1512_F22/ProjectB


## Test original dataset on model to establish upper bound

In [5]:
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image

class myArgs:
  def __init__(self):
    self.method = 'DC'
    self.dataset = 'CIFAR10'
    self.model = 'ConvNet'
    self.ipc = 10
    self.eval_mode = 'S'
    self.num_exp = 1
    self.num_eval = 5
    self.epoch_eval_train = 300
    self.Iteration = 100
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.init = 'real'
    self.dsa_strategy = 'None'
    self.data_path = 'data'
    self.save_path = 'result'
    self.dis_metric = 'ours'

args = myArgs()
args.outer_loop, args.inner_loop = get_loops(args.ipc)
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.dsa_param = ParamDiffAug()
args.dsa = True if args.method == 'DSA' else False


channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

net = get_network(args.model, channel, num_classes, im_size).to(args.device)

net.train()
optimizer_net = torch.optim.SGD(net.parameters(), lr=0.01)  # optimizer_img for synthetic data
optimizer_net.zero_grad()

criterion = nn.CrossEntropyLoss().to(args.device)

num_epochs = 20

trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, num_epochs)


for i in range(num_epochs):
  loss_avg, acc_avg = epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)
  for param_group in optimizer_net.param_groups:
    print("Current learning rate is: {}".format(param_group['lr']))

  print("Epoch[{}/{}]: Loss: {:.4f}, ACC: {:.4f}".format(i+1,num_epochs, loss_avg, acc_avg))
  scheduler.step()



Files already downloaded and verified
Files already downloaded and verified
Current learning rate is: 0.01
Epoch[1/20]: Loss: 1.8086, ACC: 0.3899
Current learning rate is: 0.009938441702975689
Epoch[2/20]: Loss: 1.4336, ACC: 0.5174
Current learning rate is: 0.009755282581475769
Epoch[3/20]: Loss: 1.2545, ACC: 0.5771
Current learning rate is: 0.00945503262094184
Epoch[4/20]: Loss: 1.1425, ACC: 0.6143
Current learning rate is: 0.009045084971874739
Epoch[5/20]: Loss: 1.0573, ACC: 0.6441
Current learning rate is: 0.008535533905932738
Epoch[6/20]: Loss: 0.9972, ACC: 0.6647
Current learning rate is: 0.007938926261462366
Epoch[7/20]: Loss: 0.9464, ACC: 0.6819
Current learning rate is: 0.007269952498697735
Epoch[8/20]: Loss: 0.9055, ACC: 0.6961
Current learning rate is: 0.006545084971874738
Epoch[9/20]: Loss: 0.8710, ACC: 0.7081
Current learning rate is: 0.005782172325201155
Epoch[10/20]: Loss: 0.8387, ACC: 0.7197
Current learning rate is: 0.005
Epoch[11/20]: Loss: 0.8159, ACC: 0.7277
Current 

In [5]:
testloader = torch.utils.data.DataLoader(dst_test, batch_size=args.batch_train, shuffle=True, num_workers=0)
net.eval()
loss_avg, acc_avg = epoch('test', testloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

print("Test set: Loss: {:.4f}, ACC: {:.4f}".format(loss_avg, acc_avg))

Test set: Loss: 0.8286, ACC: 0.7234


## Dataset distillation with matching gradient 

In [6]:
import os
import time
import copy
import argparse
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import save_image
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug


class myArgs:
  def __init__(self):
    self.method = 'DC'
    self.dataset = 'CIFAR10'
    self.model = 'ConvNet'
    self.ipc = 10
    self.eval_mode = 'S'
    self.num_exp = 1
    self.num_eval = 5
    self.epoch_eval_train = 300
    self.Iteration = 100
    self.lr_img = 0.1
    self.lr_net = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.init = 'noise'
    self.dsa_strategy = 'None'
    self.data_path = 'data'
    self.save_path = 'result'
    self.dis_metric = 'ours'

def main():

    # parser = argparse.ArgumentParser(description='Parameter Processing')
    # parser.add_argument('--method', type=str, default='DC', help='DC/DSA')
    # parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
    # parser.add_argument('--model', type=str, default='ConvNet', help='model')
    # parser.add_argument('--ipc', type=int, default=1, help='image(s) per class')
    # parser.add_argument('--eval_mode', type=str, default='S', help='eval_mode') # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
    # parser.add_argument('--num_exp', type=int, default=5, help='the number of experiments')
    # parser.add_argument('--num_eval', type=int, default=20, help='the number of evaluating randomly initialized models')
    # parser.add_argument('--epoch_eval_train', type=int, default=300, help='epochs to train a model with synthetic data')
    # parser.add_argument('--Iteration', type=int, default=1000, help='training iterations')
    # parser.add_argument('--lr_img', type=float, default=0.1, help='learning rate for updating synthetic images')
    # parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
    # parser.add_argument('--batch_real', type=int, default=256, help='batch size for real data')
    # parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
    # parser.add_argument('--init', type=str, default='noise', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
    # parser.add_argument('--dsa_strategy', type=str, default='None', help='differentiable Siamese augmentation strategy')
    # parser.add_argument('--data_path', type=str, default='data', help='dataset path')
    # parser.add_argument('--save_path', type=str, default='result', help='path to save results')
    # parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric')

    # args = parser.parse_args()
    args = myArgs()
    args.outer_loop, args.inner_loop = get_loops(args.ipc)
    args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    args.dsa_param = ParamDiffAug()
    args.dsa = True if args.method == 'DSA' else False


    if not os.path.exists(args.data_path):
        os.mkdir(args.data_path)

    if not os.path.exists(args.save_path):
        os.mkdir(args.save_path)

    eval_it_pool = np.arange(0, args.Iteration+1, 20).tolist() if args.eval_mode == 'S' or args.eval_mode == 'SS' else [args.Iteration] # The list of iterations when we evaluate models and record results.
    print('eval_it_pool: ', eval_it_pool)
    channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
    model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)


    accs_all_exps = dict() # record performances of all experiments
    for key in model_eval_pool:
        accs_all_exps[key] = []

    data_save = []


    for exp in range(args.num_exp):
        print('\n================== Exp %d ==================\n '%exp)
        print('Hyper-parameters: \n', args.__dict__)
        print('Evaluation model pool: ', model_eval_pool)

        ''' organize the real dataset '''
        images_all = []
        labels_all = []
        indices_class = [[] for c in range(num_classes)]

        images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
        labels_all = [dst_train[i][1] for i in range(len(dst_train))]
        for i, lab in enumerate(labels_all):
            indices_class[lab].append(i)
        images_all = torch.cat(images_all, dim=0).to(args.device)
        labels_all = torch.tensor(labels_all, dtype=torch.long, device=args.device)

        for c in range(num_classes):
            print('class c = %d: %d real images'%(c, len(indices_class[c])))

        def get_images(c, n): # get random n images from class c
            idx_shuffle = np.random.permutation(indices_class[c])[:n]
            return images_all[idx_shuffle]

        for ch in range(channel):
            print('real images channel %d, mean = %.4f, std = %.4f'%(ch, torch.mean(images_all[:, ch]), torch.std(images_all[:, ch])))


        ''' initialize the synthetic data '''
        image_syn = torch.randn(size=(num_classes*args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float, requires_grad=True, device=args.device)
        label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1) # [0,0,0, 1,1,1, ..., 9,9,9]

        if args.init == 'real':
            print('initialize synthetic data from random real images')
            for c in range(num_classes):
                image_syn.data[c*args.ipc:(c+1)*args.ipc] = get_images(c, args.ipc).detach().data
        else:
            print('initialize synthetic data from random noise')


        ''' training '''
        optimizer_img = torch.optim.SGD([image_syn, ], lr=args.lr_img, momentum=0.5) # optimizer_img for synthetic data
        optimizer_img.zero_grad()
        criterion = nn.CrossEntropyLoss().to(args.device)
        print('%s training begins'%get_time())

        for it in range(args.Iteration+1):

            ''' Evaluate synthetic data '''
            if it in eval_it_pool:
                for model_eval in model_eval_pool:
                    print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(args.model, model_eval, it))
                    if args.dsa:
                        args.epoch_eval_train = 1000
                        args.dc_aug_param = None
                        print('DSA augmentation strategy: \n', args.dsa_strategy)
                        print('DSA augmentation parameters: \n', args.dsa_param.__dict__)
                    else:
                        args.dc_aug_param = get_daparam(args.dataset, args.model, model_eval, args.ipc) # This augmentation parameter set is only for DC method. It will be muted when args.dsa is True.
                        print('DC augmentation parameters: \n', args.dc_aug_param)

                    if args.dsa or args.dc_aug_param['strategy'] != 'none':
                        args.epoch_eval_train = 1000  # Training with data augmentation needs more epochs.
                    else:
                        args.epoch_eval_train = 300

                    accs = []
                    for it_eval in range(args.num_eval):
                        net_eval = get_network(model_eval, channel, num_classes, im_size).to(args.device) # get a random model
                        image_syn_eval, label_syn_eval = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach()) # avoid any unaware modification
                        _, acc_train, acc_test = evaluate_synset(it_eval, net_eval, image_syn_eval, label_syn_eval, testloader, args)
                        accs.append(acc_test)
                    print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

                    if it == args.Iteration: # record the final results
                        accs_all_exps[model_eval] += accs

                ''' visualize and save '''
                save_name = os.path.join(args.save_path, 'vis_%s_%s_%s_%dipc_exp%d_iter%d.png'%(args.method, args.dataset, args.model, args.ipc, exp, it))
                image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
                for ch in range(channel):
                    image_syn_vis[:, ch] = image_syn_vis[:, ch]  * std[ch] + mean[ch]
                image_syn_vis[image_syn_vis<0] = 0.0
                image_syn_vis[image_syn_vis>1] = 1.0
                save_image(image_syn_vis, save_name, nrow=args.ipc) # Trying normalize = True/False may get better visual effects.


            ''' Train synthetic data '''
            net = get_network(args.model, channel, num_classes, im_size).to(args.device) # get a random model
            net.train()
            net_parameters = list(net.parameters())
            optimizer_net = torch.optim.SGD(net.parameters(), lr=args.lr_net)  # optimizer_img for synthetic data
            optimizer_net.zero_grad()
            loss_avg = 0
            args.dc_aug_param = None  # Mute the DC augmentation when learning synthetic data (in inner-loop epoch function) in oder to be consistent with DC paper.


            for ol in range(args.outer_loop):

                ''' freeze the running mu and sigma for BatchNorm layers '''
                # Synthetic data batch, e.g. only 1 image/batch, is too small to obtain stable mu and sigma.
                # So, we calculate and freeze mu and sigma for BatchNorm layer with real data batch ahead.
                # This would make the training with BatchNorm layers easier.

                BN_flag = False
                BNSizePC = 16  # for batch normalization
                for module in net.modules():
                    if 'BatchNorm' in module._get_name(): #BatchNorm
                        BN_flag = True
                if BN_flag:
                    img_real = torch.cat([get_images(c, BNSizePC) for c in range(num_classes)], dim=0)
                    net.train() # for updating the mu, sigma of BatchNorm
                    output_real = net(img_real) # get running mu, sigma
                    for module in net.modules():
                        if 'BatchNorm' in module._get_name():  #BatchNorm
                            module.eval() # fix mu and sigma of every BatchNorm layer


                ''' update synthetic data '''
                loss = torch.tensor(0.0).to(args.device)
                for c in range(num_classes):
                    img_real = get_images(c, args.batch_real)
                    lab_real = torch.ones((img_real.shape[0],), device=args.device, dtype=torch.long) * c
                    img_syn = image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
                    lab_syn = torch.ones((args.ipc,), device=args.device, dtype=torch.long) * c

                    if args.dsa:
                        seed = int(time.time() * 1000) % 100000
                        img_real = DiffAugment(img_real, args.dsa_strategy, seed=seed, param=args.dsa_param)
                        img_syn = DiffAugment(img_syn, args.dsa_strategy, seed=seed, param=args.dsa_param)

                    output_real = net(img_real)
                    loss_real = criterion(output_real, lab_real)
                    gw_real = torch.autograd.grad(loss_real, net_parameters)
                    gw_real = list((_.detach().clone() for _ in gw_real))

                    output_syn = net(img_syn)
                    loss_syn = criterion(output_syn, lab_syn)
                    gw_syn = torch.autograd.grad(loss_syn, net_parameters, create_graph=True)

                    loss += match_loss(gw_syn, gw_real, args)

                optimizer_img.zero_grad()
                loss.backward()
                optimizer_img.step()
                loss_avg += loss.item()

                if ol == args.outer_loop - 1:
                    break


                ''' update network '''
                image_syn_train, label_syn_train = copy.deepcopy(image_syn.detach()), copy.deepcopy(label_syn.detach())  # avoid any unaware modification
                dst_syn_train = TensorDataset(image_syn_train, label_syn_train)
                trainloader = torch.utils.data.DataLoader(dst_syn_train, batch_size=args.batch_train, shuffle=True, num_workers=0)
                for il in range(args.inner_loop):
                    epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)


            loss_avg /= (num_classes*args.outer_loop)

            if it%10 == 0:
                print('%s iter = %04d, loss = %.4f' % (get_time(), it, loss_avg))

            if it == args.Iteration: # only record the final results
                data_save.append([copy.deepcopy(image_syn.detach().cpu()), copy.deepcopy(label_syn.detach().cpu())])
                torch.save({'data': data_save, 'accs_all_exps': accs_all_exps, }, os.path.join(args.save_path, 'res_%s_%s_%s_%dipc.pt'%(args.method, args.dataset, args.model, args.ipc)))


    print('\n==================== Final Results ====================\n')
    for key in model_eval_pool:
        accs = accs_all_exps[key]
        print('Run %d experiments, train on %s, evaluate %d random %s, mean  = %.2f%%  std = %.2f%%'%(args.num_exp, args.model, len(accs), key, np.mean(accs)*100, np.std(accs)*100))


In [7]:
main()

eval_it_pool:  [0, 20, 40, 60, 80, 100]
Files already downloaded and verified
Files already downloaded and verified

================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DC', 'dataset': 'CIFAR10', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 5, 'epoch_eval_train': 300, 'Iteration': 100, 'lr_img': 0.1, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'noise', 'dsa_strategy': 'None', 'data_path': 'data', 'save_path': 'result', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f8308cbcc50>, 'dsa': False}
Evaluation model pool:  ['ConvNet']
class c = 0: 5000 real images
class c = 1: 5000 real images
class c = 2: 5000 real images
class c = 3: 5000 real images
class c = 4: 5000 real images
class c = 5: 5000 real images
class c = 6: 5000 real images
class c = 7: 5000 real images
class c = 8: 5000 real images
class c = 9: 5000 real images
real images chan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


[2022-11-28 15:35:30] Evaluate_00: epoch = 0300 train time = 3 s train loss = 0.003668 train acc = 1.0000, test acc = 0.1075
[2022-11-28 15:35:37] Evaluate_01: epoch = 0300 train time = 3 s train loss = 0.003676 train acc = 1.0000, test acc = 0.0826
[2022-11-28 15:35:43] Evaluate_02: epoch = 0300 train time = 3 s train loss = 0.003784 train acc = 1.0000, test acc = 0.1050
[2022-11-28 15:35:49] Evaluate_03: epoch = 0300 train time = 3 s train loss = 0.003776 train acc = 1.0000, test acc = 0.0990
[2022-11-28 15:35:55] Evaluate_04: epoch = 0300 train time = 3 s train loss = 0.003880 train acc = 1.0000, test acc = 0.0911
Evaluate 5 random ConvNet, mean = 0.0970 std = 0.0092
-------------------------
[2022-11-28 15:36:06] iter = 0000, loss = 225.0190
[2022-11-28 15:37:43] iter = 0010, loss = 148.2568
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 20
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strategy'

In [ ]:
#main()

eval_it_pool:  [0, 20, 40, 60, 80, 100]


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Attention: Here I will replace BN with IN in evaluation, as the synthetic set is too small to measure BN hyper-parameters.

================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DC', 'dataset': 'CIFAR10', 'model': 'ResNet18BN_AP', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 5, 'epoch_eval_train': 300, 'Iteration': 100, 'lr_img': 0.1, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'real', 'dsa_strategy': 'None', 'data_path': 'data', 'save_path': 'result', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7f8a44d56e50>, 'dsa': False}
Evaluation model pool:  ['ResNet18']
class c = 0: 5000 real images
class c = 1: 5000 real images
class c = 2: 5000 real images
class c = 3: 5000 real images
class c = 4: 5000 real images
class c = 5: 5000 real images
class c = 6: 5000 real images
class c 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


[2022-11-26 03:01:56] Evaluate_00: epoch = 0300 train time = 33 s train loss = 0.005744 train acc = 1.0000, test acc = 0.1986
[2022-11-26 03:02:29] Evaluate_01: epoch = 0300 train time = 27 s train loss = 0.005755 train acc = 1.0000, test acc = 0.2089
[2022-11-26 03:03:02] Evaluate_02: epoch = 0300 train time = 28 s train loss = 0.005846 train acc = 1.0000, test acc = 0.1960
[2022-11-26 03:03:35] Evaluate_03: epoch = 0300 train time = 28 s train loss = 0.005902 train acc = 1.0000, test acc = 0.2062
[2022-11-26 03:04:09] Evaluate_04: epoch = 0300 train time = 29 s train loss = 0.005653 train acc = 1.0000, test acc = 0.2030
Evaluate 5 random ResNet18, mean = 0.2025 std = 0.0047
-------------------------
[2022-11-26 03:05:30] iter = 0000, loss = 1895.7798
[2022-11-26 03:18:59] iter = 0010, loss = 2683.2079
-------------------------
Evaluation
model_train = ResNet18BN_AP, model_eval = ResNet18, iteration = 20
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.

In [ ]:
main()

eval_it_pool:  [0, 20, 40, 60, 80, 100]

================== Exp 0 ==================
 
Hyper-parameters: 
 {'method': 'DC', 'dataset': 'MNIST', 'model': 'ConvNet', 'ipc': 10, 'eval_mode': 'S', 'num_exp': 1, 'num_eval': 5, 'epoch_eval_train': 300, 'Iteration': 100, 'lr_img': 0.1, 'lr_net': 0.01, 'batch_real': 256, 'batch_train': 256, 'init': 'noise', 'dsa_strategy': 'None', 'data_path': 'data', 'save_path': 'result', 'dis_metric': 'ours', 'outer_loop': 10, 'inner_loop': 50, 'device': 'cuda', 'dsa_param': <utils.ParamDiffAug object at 0x7fc13e328210>, 'dsa': False}
Evaluation model pool:  ['ConvNet']
class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
real images channel 0, mean = -0.0001, std = 1.0000
initialize synthetic data from random nois

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:112: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


[2022-11-27 16:31:00] Evaluate_00: epoch = 1000 train time = 34 s train loss = 0.012398 train acc = 1.0000, test acc = 0.0855
[2022-11-27 16:31:37] Evaluate_01: epoch = 1000 train time = 35 s train loss = 0.008104 train acc = 1.0000, test acc = 0.0779
[2022-11-27 16:32:14] Evaluate_02: epoch = 1000 train time = 34 s train loss = 0.019059 train acc = 1.0000, test acc = 0.0687
[2022-11-27 16:32:51] Evaluate_03: epoch = 1000 train time = 34 s train loss = 0.010604 train acc = 1.0000, test acc = 0.0666
[2022-11-27 16:33:28] Evaluate_04: epoch = 1000 train time = 34 s train loss = 0.010578 train acc = 1.0000, test acc = 0.0496
Evaluate 5 random ConvNet, mean = 0.0697 std = 0.0121
-------------------------
[2022-11-27 16:33:37] iter = 0000, loss = 221.2675
[2022-11-27 16:35:10] iter = 0010, loss = 93.1256
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 20
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strat

In [8]:
#!zip -r /content/result.zip /content/result
#from google.colab import files
#files.download("/content/result.zip")

	zip warning: name not matched: /content/result

zip error: Nothing to do! (try: zip -r /content/result.zip . -i /content/result)


FileNotFoundError: ignored

## Load up distilled dataset and train model 

In [7]:
distilled_set_name = 'result/res_DC_CIFAR10_ConvNet_10ipc.pt'
dataset = torch.load(distilled_set_name)
syn_data = dataset['data']
syn_data = TensorDataset(syn_data[0][0], syn_data[0][1])

channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)
net = get_network(args.model, channel, num_classes, im_size).to(args.device)


net.train()
optimizer_net = torch.optim.SGD(net.parameters(), lr=0.01)  # optimizer_img for synthetic data
optimizer_net.zero_grad()

criterion = nn.CrossEntropyLoss().to(args.device)

num_epochs = 20

trainloader = torch.utils.data.DataLoader(syn_data, batch_size=1, shuffle=True, num_workers=0)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, num_epochs)


for i in range(num_epochs):
  loss_avg, acc_avg = epoch('train', trainloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)
  for param_group in optimizer_net.param_groups:
    print("Current learning rate is: {}".format(param_group['lr']))

  print("Epoch[{}/{}]: Loss: {:.4f}, ACC: {:.4f}".format(i+1,num_epochs, loss_avg, acc_avg))
  scheduler.step()


testloader = torch.utils.data.DataLoader(dst_test, batch_size=args.batch_train, shuffle=True, num_workers=0)
net.eval()
loss_avg, acc_avg = epoch('test', testloader, net, optimizer_net, criterion, args, aug = True if args.dsa else False)

print("Test set: Loss: {:.4f}, ACC: {:.4f}".format(loss_avg, acc_avg))

Files already downloaded and verified
Files already downloaded and verified
Current learning rate is: 0.01
Epoch[1/20]: Loss: 4.4897, ACC: 0.1200
Current learning rate is: 0.009938441702975689
Epoch[2/20]: Loss: 3.0855, ACC: 0.2200
Current learning rate is: 0.009755282581475769
Epoch[3/20]: Loss: 2.2197, ACC: 0.3100
Current learning rate is: 0.00945503262094184
Epoch[4/20]: Loss: 1.6446, ACC: 0.4100
Current learning rate is: 0.009045084971874739
Epoch[5/20]: Loss: 0.8483, ACC: 0.6900
Current learning rate is: 0.008535533905932738
Epoch[6/20]: Loss: 0.4707, ACC: 0.8700
Current learning rate is: 0.007938926261462366
Epoch[7/20]: Loss: 0.2005, ACC: 0.9800
Current learning rate is: 0.007269952498697735
Epoch[8/20]: Loss: 0.0928, ACC: 0.9900
Current learning rate is: 0.006545084971874738
Epoch[9/20]: Loss: 0.0513, ACC: 1.0000
Current learning rate is: 0.005782172325201155
Epoch[10/20]: Loss: 0.0395, ACC: 1.0000
Current learning rate is: 0.005
Epoch[11/20]: Loss: 0.0338, ACC: 1.0000
Current 